In [1]:
from pix2text import Pix2Text
from text_recognizer import TextRecognizerRus
from PIL import Image

In [2]:
img_fp = 'data/image.png'
image = Image.open(img_fp)


In [3]:
p2t = Pix2Text(analyzer_config=dict(model_name='mfd'))
text_rec = TextRecognizerRus()

In [4]:
outs = p2t.recognize(img_fp, resized_shape=608, return_text=False)

[WARNING 2024-03-20 00:31:27,477 _showwarnmsg:109] /home/vlad/park/LatexOCR/venv/lib/python3.11/site-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
 


100%|██████████| 11/11 [00:05<00:00,  1.89it/s]


In [5]:
line_number = 0
markdown = ''
for element in outs:
    text = element['text']
    if element['type'] == 'text':
        top_left = element['position'][0]
        bottom_right = element['position'][2]
        coords = tuple(list(top_left) + list(bottom_right))
        cropped = image.crop(coords)
        cropped.save('cropped.png')
        text = text_rec.recognize('cropped.png').strip()
    
    if element['type'] == 'embedding':
        text = '\n`$$' + text + '$$`\n'
    if element['type'] == 'isolated':
        text = '```KaTeX\n' + text + '\n```' 

    if element['line_number'] == line_number:
        markdown += text
    else:
        markdown += '\n' + text
        line_number = element['line_number']
print(markdown)

Тогда решение для
`$$x_{i-1}$$`
может быть выражено через
`$$x_{i}$$`

```KaTeX
x_{i-1}=\frac{\tilde{b}_{i-1}-\tilde{a}_{i-1, i} x_{i}} {\tilde{a}_{i-1, i-1}}
```
Можно представить эту запись в упрощенном виде:
```KaTeX
x_{i-1}=\gamma_{i} x_{i}+\beta_{i},
```
где для коэффициентов
`$$\gamma$$`

`$$\beta$$`
существует следующие рекуррентные соотношения:
```KaTeX
\gamma_{i+1}=\frac{-a_{i, i+1}} {a_{i, i-1} \gamma_{i}+a_{i i}}, \tag{3}
```
```KaTeX
\beta_{i+1}=\frac{b_{i}-a_{i, i-1} \beta_{i}} {a_{i, i-1} \gamma_{i}+a_{i i}}. \tag{4}
```
Начальные коэффициенты
`$$\gamma_{1}$$`

`$$\beta_{1}$$`
выражаются из первого уравнения СЛАУ (2):
```KaTeX
\begin{aligned} {\gamma_{1}} & {{}=\frac{-a_{1 2}} {a_{1 1}},} \\ {\beta_{1}} & {{}=\frac{b_{1}} {a_{1 1}}.} \\ \end{aligned}
```
